# Combinatory Categorial Grammars in GF

 ## Basics
 


 * Few Basic Types (N,S,NP)
 * Directed Function Types $(X/Y),(X\backslash Y)$
 * Complex Types in the Lexicon, e.g. $bite\ :\ (S\backslash NP)/NP$

## Combinators

 * Function Application: $$>:\frac{\alpha : X/Y\quad \beta : Y}{\alpha\beta : X}$$,$$<:\frac{\beta : X\quad \alpha : Y\backslash X}{\beta\alpha : Y}$$
 * Function Composition $$B_>: \frac{\alpha : X/Y\quad \beta : Y/Z}{\alpha\beta : X/Z}$$,$$B_<:\frac{\beta : Y\backslash Z\quad \alpha : X\backslash Y}{\beta\alpha X\backslash Z}$$
 * Type Raising $$T_>: \frac{\alpha : X}{\alpha : T/(T\backslash X)}$$,$$T_<: \frac{\alpha : X}{\alpha : T
 \backslash(T/ X)}$$

## Abstract Syntax

In [1]:
abstract CGAbs = {
  flags startcat = Sentence ;
  cat Cat ;
      CGType ;
      CG CGType ;
      Sentence ;
  data
    Single : Cat -> CGType ;
    FSlash : CGType -> CGType -> CGType ;
    BSlash : CGType -> CGType -> CGType ;

  fun
    -- To form startcat
    complete : CG (Single S) -> Sentence ;
    -- The categories
    N : Cat ;
    S : Cat ;
    NP : Cat ;
    -- The combinators
    -- Function composition
    FB : ({x,y,z} : CGType) -> CG (FSlash x y) -> CG (FSlash y z) -> CG (FSlash x z) ;
    BB : ({z,y,x} : CGType) -> CG (BSlash z y) -> CG (BSlash y x) -> CG (BSlash z x) ;
    -- Function application
    FS : ({x,y} : CGType) -> CG (FSlash x y) -> CG y -> CG x ;
    BS : ({y,x} : CGType) -> CG y -> CG (BSlash x y) -> CG x ;
    -- Type raising
    -- X -> T -> T/(T\X)
    FT : ({x,t} : CGType) -> CG x -> CG (FSlash t (BSlash t x)) ;
    -- X -> T -> T\(T/X)
    BT : ({x,t} : CGType) -> CG x -> CG (BSlash t (FSlash t x)) ;

    -- The lexicon
    -- man : CG (Single N) ;
    -- sleeps : CG (BSlash (Single N) (Single S)) ;
    -- john : CG (Single N) ;
    -- mary : CG (Single N) ;
    -- loves : CG (FSlash (BSlash (Single N) (Single S)) (Single N));
    -- the : CG (FSlash (Single N) (Single N));
    -- that : CG (FSlash (Single N) (Single N));
    -- small : CG (FSlash (Single N) (Single N));
    -- hungry : CG (FSlash (Single N) (Single N));
    -- brown : CG (FSlash (Single N) (Single N));
    -- dog : CG (Single N) ;
    the : CG (FSlash (Single NP) (Single N)) ;
    dog : CG (Single N) ; 
    bit : CG (FSlash (BSlash (Single S) (Single NP)) (Single NP)) ;
    john : CG (Single NP) ;

}

/tmp/tmprkfcryh2/CGAbs.gf


Defined CGAbs

### Example Derivation

Sentence: the dog bit john

$$\frac{\frac{the:NP/N\quad dog:N}{the\ dog:NP} >\quad\frac{bit:(NP\backslash S)/NP\quad john:NP}{bit\ john:NP\backslash S}>}{the\ dog\ bit\ john:S}>$$

In [2]:
ai dog

fun dog : CG (Single N) ;
Probability: 0.1


In [3]:
ai the

fun the : CG (FSlash (Single NP) (Single N)) ;
Probability: 0.1


$$ \frac{the:NP/N\quad dog:N}{the\ dog:NP} >$$

In [4]:
ai (FS the dog)

Expression:  FS {Single NP} {Single N} the dog
Type:        CG (Single NP)
Probability: 1.0000000000000002e-3


$$\frac{bit:(NP\backslash S)/NP\quad john:NP}{bit\ john:NP\backslash S}>$$

In [5]:
ai (FS bit john)

Expression:  FS {BSlash (Single S) (Single NP)} {Single NP} bit john
Type:        CG (BSlash (Single S) (Single NP))
Probability: 1.0000000000000002e-3


$$\frac{the\ dog:NP\quad bit\ john:NP\backslash S}{the\ dog\ bit\ john:S}>$$

In [6]:
ai (BS (FS the dog) (FS bit john))

Expression:  BS {Single NP} {Single S} (FS {Single NP} {Single N} the dog) (FS {BSlash (Single S) (Single NP)} {Single NP} bit john)
Type:        CG (Single S)
Probability: 1.0000000000000006e-7


Derivation using type raisng and function composition

In [7]:
ai (FS the dog)

Expression:  FS {Single NP} {Single N} the dog
Type:        CG (Single NP)
Probability: 1.0000000000000002e-3


Type raising

In [8]:
ai (FT (FS the dog))

Expression:  FT {Single NP} {?2} (FS {Single NP} {Single N} the dog)
Type:        CG (FSlash ?2 (BSlash ?2 (Single NP)))
Probability: 1.0000000000000003e-4


Function composition

In [9]:
ai (FB (FT (FS the dog)) bit)

Expression:  FB {Single S} {BSlash (Single S) (Single NP)} {Single NP} (FT {Single NP} {Single S} (FS {Single NP} {Single N} the dog)) bit
Type:        CG (FSlash (Single S) (Single NP))
Probability: 1.0000000000000004e-6


In [10]:
ai (FS (FB (FT (FS the dog)) bit) john)

Expression:  FS {Single S} {Single NP} (FB {Single S} {BSlash (Single S) (Single NP)} {Single NP} (FT {Single NP} {Single S} (FS {Single NP} {Single N} the dog)) bit) john
Type:        CG (Single S)
Probability: 1.0000000000000005e-8


### Problem

In [ ]:
gt -cat="IsEqual ? z"

## Concrete Syntax

### Boilerplate

In [11]:
incomplete concrete CGFun of CGAbs = open Prelude, Predef in {
  lincat
    Cat, CGType, CG = Str ;
    Sentence = SS ;
  lin
    N = "" ;
    S = "" ;
    NP = "" ;
    Single c = c ;
    FSlash c1 c2 = c1 ++ c2 ;
    BSlash c2 c1 = c2 ++ c1 ;
    
    FB x y z xy yz = x ++ y ++ z ++ xy ++ yz ;
    BB x y z zy yx = x ++ y ++ z ++ zy ++ yx ;
    FS x y xy y2 = x ++ y ++ xy ++ y2 ;
    BS x y y2 yx = x ++ y ++ y2 ++ yx ;
    FT x y x2 = x ++ y ++ x2 ;
    BT x y x2 = x ++ y ++ x2 ;
    complete s = ss s ;
}

/tmp/tmpcz186jwa/CGFun.gf


Defined CGFun

### English

In [12]:
concrete CGEng of CGAbs = CGFun with (Prelude=Prelude),(Predef=Predef) ** {
  lin
    -- man = "man" ;
    -- sleeps = "sleeps" ;
    -- john = "John" ;
    -- mary = "Mary" ;
    -- loves = "loves" ;
    the = "the" ;
    -- that = "that" ;
    -- small = "small" ;
    -- hungry = "hungry" ;
    -- brown = "brown" ;
    dog = "dog" ;
    bit = "bit" ;
    john = "John" ;
}

/tmp/tmpcz186jwa/CGEng.gf


Defined CGEng

In [13]:
l (BS (FS the dog) (FS bit john))

the dog bit John


In [14]:
l (FS (FB (FT (FS the dog)) bit) john)

the dog bit John


### Problem

In [ ]:
p -cat=Sentence -lang=CGEng "the dog bit John"

```
CGAbs> p -cat=Sentence "the dog bit John"
The parsing is successful but the type checking failed with error(s):
  Couldn't match expected type CG (BSlash (Single N) (Single N))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (BSlash (Single N) (Single NP))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (BSlash (Single N) (Single S))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (BSlash (Single NP) (Single N))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (BSlash (Single NP) (Single NP))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (BSlash (Single NP) (Single S))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (BSlash (Single S) (Single N))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (BSlash (Single S) (Single NP))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (BSlash (Single S) (Single S))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (Single N)
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (Single NP)
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (Single S)
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (FSlash (Single N) (Single N))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (FSlash (Single N) (Single NP))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (FSlash (Single N) (Single S))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (FSlash (Single N) (Single N))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (FSlash (Single N) (Single NP))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (FSlash (Single N) (Single S))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (FSlash (Single NP) (Single NP))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (FSlash (Single NP) (Single S))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (FSlash (Single S) (Single N))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (FSlash (Single S) (Single NP))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (FSlash (Single S) (Single S))
         against inferred type CG (FSlash (Single NP) (Single N))
  In the expression: the
  Couldn't match expected type CG (BSlash (Single N) (Single N))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (BSlash (Single N) (Single NP))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (BSlash (Single N) (Single S))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (BSlash (Single NP) (Single N))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (BSlash (Single NP) (Single NP))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (BSlash (Single NP) (Single S))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (BSlash (Single S) (Single N))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (BSlash (Single S) (Single NP))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (BSlash (Single S) (Single S))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (Single N)
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (BSlash (Single N) (Single NP))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (BSlash (Single NP) (Single NP))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (BSlash (Single S) (Single NP))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (Single S)
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (BSlash (Single N) (Single N))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (BSlash (Single N) (Single NP))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (BSlash (Single N) (Single S))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (BSlash (Single NP) (Single N))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (BSlash (Single NP) (Single NP))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (BSlash (Single NP) (Single S))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (BSlash (Single S) (Single N))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (BSlash (Single S) (Single NP))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (BSlash (Single S) (Single S))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (BSlash (Single N) (Single N))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (BSlash (Single NP) (Single N))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (BSlash (Single S) (Single N))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (Single NP)
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (Single S)
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (FSlash (Single NP) (Single N))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (FSlash (Single NP) (Single NP))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (FSlash (Single NP) (Single S))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (FSlash (Single S) (Single N))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (FSlash (Single S) (Single NP))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (FSlash (Single S) (Single S))
         against inferred type CG (Single N)
  In the expression: dog
  Couldn't match expected type CG (FSlash (Single N) (Single N))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (FSlash (Single N) (Single NP))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (FSlash (Single N) (Single S))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (FSlash (Single NP) (Single N))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (FSlash (Single NP) (Single NP))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (FSlash (Single NP) (Single S))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (FSlash (Single S) (Single N))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (FSlash (Single S) (Single NP))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (FSlash (Single S) (Single S))
         against inferred type CG (Single NP)
  In the expression: FS {Single NP} {Single N} the dog
  Couldn't match expected type CG (BSlash (Single N) (Single S))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (BSlash (Single NP) (Single S))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (BSlash (Single S) (Single S))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (Single N)
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (Single NP)
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (Single S)
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (FSlash (Single N) (Single N))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (FSlash (Single N) (Single NP))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (FSlash (Single N) (Single S))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (FSlash (Single NP) (Single N))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (FSlash (Single NP) (Single NP))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (FSlash (Single NP) (Single S))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (FSlash (Single S) (Single N))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (FSlash (Single S) (Single NP))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
  Couldn't match expected type CG (FSlash (Single S) (Single S))
         against inferred type CG (FSlash (BSlash (Single S) (Single NP)) (Single NP))
  In the expression: bit
The parser failed at token 1: "the"
133118 msec
```